In [57]:
import pandas as pd
import numpy as np
import nltk##lemmatization için lazım 
from nltk.corpus import stopwords
from vnlp import DependencyParser,PoSTagger,StopwordRemover, StemmerAnalyzer
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import string
from sklearn.ensemble import RandomForestRegressor
import re
import zipfile

from jpype import JClass, getDefaultJVMPath, shutdownJVM, startJVM, JString

In [56]:
pip install JPype1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 11.7 MB/s eta 0:00:00


In [50]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [53]:
zip_ref = zipfile.ZipFile("/content/drive/My Drive/nlp/zip.zip", 'r')
zip_ref.extractall("/content")
zip_ref.close()

Dataset

In [59]:
#df = pd.read_json("./trsum/train.jsonl",lines=True)
#df=df.drop(columns=["summary","url"])
df= pd.read_csv("data_result_added",sep="\t")
df

,news_id,sentence,title,is_contains_idiom,NER,tfidf,noun_phrase,words2,sentence_length,position,result
0,1064.0,Nikah yerinizi nasıl seçersiniz?,Evlilik başvurusu nasıl yapılır?,-1,0,0,1,"['Nikah', 'yerinizi', 'nasıl', 'seçersiniz']",0.010610,0.333333,0.489865
1,1064.0,Kimler nikah şahidi seçilebilir?,Evlilik başvurusu nasıl yapılır?,-1,0,1,1,"['Kimler', 'nikah', 'şahidi', 'seçilebilir']",0.010610,0.250000,0.521200
2,1064.0,Nikahla ilgili merak ettiğiniz her şey burada…...,Evlilik başvurusu nasıl yapılır?,-1,0,0,0,"['Nikahla', 'ilgili', 'merak', 'ettiğiniz', 'h...",0.029178,0.200000,0.781842
3,1064.0,Müstakbel eşinizle birlikte yaşadığınız yerdek...,Evlilik başvurusu nasıl yapılır?,-1,1,1,0,"['Müstakbel', 'eşinizle', 'birlikte', 'yaşadığ...",0.023873,0.166667,0.657137
4,1064.0,"Ancak, ayrı illerde oturuyorsanız kendi illeri...",Evlilik başvurusu nasıl yapılır?,-1,0,1,0,"['Ancak', 'ayrı', 'illerde', 'oturuyorsanız', ...",0.031830,0.142857,0.484427
...,...,...,...,...,...,...,...,...,...,...,...
11387,1697.0,"Galatasaray, Gaziantepspor maçının hazırlıklar...",Floryada Lucas Neill rüzgarı,-1,2,0,0,"['Galatasaray', 'Gaziantepspor', 'maçının', 'h...",0.042254,0.076923,0.073290
11388,1698.0,"Başbakan Recep Tayyip Erdoğan, özel bir haber ...",Erdoğan bir dava daha kazandı,-1,3,1,3,"['Başbakan', 'Recep', 'Tayyip', 'Erdoğan', 'öz...",0.200000,1.000000,0.404898
11389,1698.0,Ankara 3.,Erdoğan bir dava daha kazandı,-1,0,0,0,"['Ankara', '3']",0.013793,0.500000,0.271632
11390,1698.0,Asliye Hukuk Mahkemesi'nde görülen davanın kar...,Erdoğan bir dava daha kazandı,-1,5,1,2,"['Asliye', 'Hukuk', 'Mahkemesinde', 'görülen',...",0.117241,0.333333,0.568497


In [ ]:
#df[0:2501].to_csv("sample2500", sep='\t', encoding='utf-8',index=False)

In [11]:
translator = str.maketrans('', '', string.punctuation)
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
d = {'news_id': [], 'sentence': [],"words":[],"title":[]}
data = pd.DataFrame(data=d)
data

,news_id,sentence,words,title


In [ ]:
c=0
for i in df.text:
    arr=sent_tokenize(i)
    for j in arr:
        dicta = {'news_id':c,'sentence': j,"words":[nltk.word_tokenize(j.translate(translator).replace("“","").replace("’",""))],"title":df.loc[c,"title"]}
        result = pd.DataFrame(dicta)
        data = pd.concat([data,result],ignore_index=True)
    c+=1


In [ ]:
data

,news_id,sentence,words,title
0,0.0,Araç sahipleri için pahalı benzine karşı yapıl...,"[Araç, sahipleri, için, pahalı, benzine, karşı...",Aracınızda yüzde 30 tarassuf edin
1,0.0,"Shell, Total, BP gibi sektörün önde gelen şirk...","[Shell, Total, BP, gibi, sektörün, önde, gelen...",Aracınızda yüzde 30 tarassuf edin
2,0.0,Seyir halinde iken gereksiz klima çalıştırmama...,"[Seyir, halinde, iken, gereksiz, klima, çalışt...",Aracınızda yüzde 30 tarassuf edin
3,0.0,"Daha az yakıt tüketimi, ilk olarak araç bakıml...","[Daha, az, yakıt, tüketimi, ilk, olarak, araç,...",Aracınızda yüzde 30 tarassuf edin
4,0.0,Bunun için motor ve lastik bakımları öne çıkıyor.,"[Bunun, için, motor, ve, lastik, bakımları, ön...",Aracınızda yüzde 30 tarassuf edin
...,...,...,...,...
46156,2500.0,Sizin söylediğiniz şu; Ahmet Tezcan:Sizin kura...,"[Sizin, söylediğiniz, şu, Ahmet, TezcanSizin, ...",Nuray Mert Allah deyip saydırdı
46157,2500.0,Nuray Mert: Olabilirim zaten bende hükümet kur...,"[Nuray, Mert, Olabilirim, zaten, bende, hüküme...",Nuray Mert Allah deyip saydırdı
46158,2500.0,Kurmuyoruz efendim.,"[Kurmuyoruz, efendim]",Nuray Mert Allah deyip saydırdı
46159,2500.0,Beni karşılaştırmayın bunları.,"[Beni, karşılaştırmayın, bunları]",Nuray Mert Allah deyip saydırdı


In [9]:
stopword_remover = StopwordRemover()


In [ ]:
"""df_idiom = pd.read_csv("zip/turkish_idioms_and_proverbs.csv",sep=";")
df_idiom=df_idiom.drop(columns=["Unnamed: 1"])
df_idiom"""

,proverbs
0,aba altında er yatar
1,"aba vakti yaba, yaba vakti aba"
2,abanın kadri yağmurda bilinir
3,"abdal ata binince bey oldum sanır, şalgam aşa ..."
4,"abdal düğünden, çocuk oyundan usanmaz"
...,...
2390,züğürt bezirgân eski defterleri karıştırır
2391,züğürt olup düşmektense uyuz olup kaşınmak yeğdir
2392,"züğürtleyen bezirgân, eski defterleri yoklar"
2393,"züğürtlük, zadeliği bozar"


In [ ]:
"""df_idiom["words"]=df_idiom["proverbs"].astype(str)
k=0
stopword_remover = StopwordRemover()
analyzer = zeyrek.MorphAnalyzer()
for j in df_idiom.proverbs:
    analyzer.lemmatize(i)[0][1][0]
    arr=stopword_remover.drop_stop_words(nltk.word_tokenize(df_idiom.loc[k,"proverbs"].translate(translator)))
    for a in range(len(arr)):
        arr[a]=analyzer.lemmatize(arr[a])[0][1][0]
    df_idiom.loc[k,"words"]=' '.join(arr)
    k+=1
#stemmer = StemmerAnalyzer()
#stemmer.predict("züğürt olup düşmektense uyuz olup kaşınmak yeğdir")"""

In [ ]:
df_idiom=pd.read_csv("idiom_final.csv",sep="\t")
df_idiom

In [ ]:
data["is_contains_idiom"]=-1
t=0
for ind in data.words:
    temp=ind.copy()
    for j in range(len(temp)):
        temp[j]=analyzer.lemmatize(temp[j])[0][1][0]
    sent=" ".join(temp)
    for j in range(len(df_idiom.words)):
        if df_idiom.words[j] in sent:
            data[t,"is_contains_idiom"]=j
        

In [ ]:
data=pd.read_csv("data",sep="\t")
#data.to_csv("data", sep='\t', encoding='utf-8',index=False)
data


,news_id,sentence,words,title,is_contains_idiom
0,0.0,Araç sahipleri için pahalı benzine karşı yapıl...,"['Araç', 'sahipleri', 'için', 'pahalı', 'benzi...",Aracınızda yüzde 30 tarassuf edin,-1
1,0.0,"Shell, Total, BP gibi sektörün önde gelen şirk...","['Shell', 'Total', 'BP', 'gibi', 'sektörün', '...",Aracınızda yüzde 30 tarassuf edin,-1
2,0.0,Seyir halinde iken gereksiz klima çalıştırmama...,"['Seyir', 'halinde', 'iken', 'gereksiz', 'klim...",Aracınızda yüzde 30 tarassuf edin,-1
3,0.0,"Daha az yakıt tüketimi, ilk olarak araç bakıml...","['Daha', 'az', 'yakıt', 'tüketimi', 'ilk', 'ol...",Aracınızda yüzde 30 tarassuf edin,-1
4,0.0,Bunun için motor ve lastik bakımları öne çıkıyor.,"['Bunun', 'için', 'motor', 've', 'lastik', 'ba...",Aracınızda yüzde 30 tarassuf edin,-1
...,...,...,...,...,...
143,9.0,Günde 100-150 bin adet üretim yapan endüstriye...,"['Günde', '100150', 'bin', 'adet', 'üretim', '...",İstanbulda 3 fırından birisi ruhsatsız,-1
144,9.0,diye konuştu.,"['diye', 'konuştu']",İstanbulda 3 fırından birisi ruhsatsız,-1
145,9.0,Türkiye'de her yıl 12 milyon ton civarında ekm...,"['Türkiyede', 'her', 'yıl', '12', 'milyon', 't...",İstanbulda 3 fırından birisi ruhsatsız,-1
146,9.0,İstanbul Fırıncılar Odası'nın verdiği rakamlar...,"['İstanbul', 'Fırıncılar', 'Odasının', 'verdiğ...",İstanbulda 3 fırından birisi ruhsatsız,-1


NER DETECTION

In [58]:
# path to jar file 
ZEMBEREK_PATH = r"./zip/zemberek-full.jar"
# start jvm
startJVM(getDefaultJVMPath(), "-ea", "-Djava.class.path=%s" % (ZEMBEREK_PATH),convertStrings=False)
TurkishMorphology: JClass = JClass("zemberek.morphology.TurkishMorphology")
PerceptronNer: JClass = JClass("zemberek.ner.PerceptronNer")
Paths: JClass = JClass("java.nio.file.Paths")
modelRoot = Paths.get('zip/test-model/model')
morphology = TurkishMorphology.createWithDefaults()
ner = PerceptronNer.loadModel(modelRoot, morphology)

In [82]:
print(df.loc[168,"sentence"])

79. dakikada Uğur'un sağdan ortasında Joao'nun kafa vuruşu az farkla yandan auta çıktı.


In [81]:
print
result =ner.findNamedEntities(JString(str(df.loc[168,"sentence"])))
temp=result.getNamedEntities();
print(temp)
for 


[[PERSON Uğur'un], [PERSON Joao'nun]]


In [ ]:
def entities_in_string(a_string):
    result = ner.findNamedEntities(JString(str(a_string.replace('i', 'İ').upper())))
    namedEntities = result.getNamedEntities();
    return namedEntities#nerin fonksiyon hali

In [ ]:
data["NER"]=0
t=0;
for i in data.sentence:
    a=entities_in_string(i)
    data.loc[t,"NER"]=len(a)
    t+=1

In [ ]:
data


,news_id,sentence,words,title,is_contains_idiom,NER
0,0.0,Araç sahipleri için pahalı benzine karşı yapıl...,"['Araç', 'sahipleri', 'için', 'pahalı', 'benzi...",Aracınızda yüzde 30 tarassuf edin,-1,0
1,0.0,"Shell, Total, BP gibi sektörün önde gelen şirk...","['Shell', 'Total', 'BP', 'gibi', 'sektörün', '...",Aracınızda yüzde 30 tarassuf edin,-1,4
2,0.0,Seyir halinde iken gereksiz klima çalıştırmama...,"['Seyir', 'halinde', 'iken', 'gereksiz', 'klim...",Aracınızda yüzde 30 tarassuf edin,-1,1
3,0.0,"Daha az yakıt tüketimi, ilk olarak araç bakıml...","['Daha', 'az', 'yakıt', 'tüketimi', 'ilk', 'ol...",Aracınızda yüzde 30 tarassuf edin,-1,0
4,0.0,Bunun için motor ve lastik bakımları öne çıkıyor.,"['Bunun', 'için', 'motor', 've', 'lastik', 'ba...",Aracınızda yüzde 30 tarassuf edin,-1,1
...,...,...,...,...,...,...
143,9.0,Günde 100-150 bin adet üretim yapan endüstriye...,"['Günde', '100150', 'bin', 'adet', 'üretim', '...",İstanbulda 3 fırından birisi ruhsatsız,-1,1
144,9.0,diye konuştu.,"['diye', 'konuştu']",İstanbulda 3 fırından birisi ruhsatsız,-1,0
145,9.0,Türkiye'de her yıl 12 milyon ton civarında ekm...,"['Türkiyede', 'her', 'yıl', '12', 'milyon', 't...",İstanbulda 3 fırından birisi ruhsatsız,-1,2
146,9.0,İstanbul Fırıncılar Odası'nın verdiği rakamlar...,"['İstanbul', 'Fırıncılar', 'Odasının', 'verdiğ...",İstanbulda 3 fırından birisi ruhsatsız,-1,2


In [ ]:
def tfidf(thres):
    data["tfidf"]=0
    ind=0
    ind2=0
    while ind<len(data.sentence):
        id=data.loc[ind,"news_id"]
        metin=ret_text(ind)
        vectorizer = TfidfVectorizer()
        X = vectorizer.fit_transform([metin])
        X.toarray()
        tfidf_dict = dict(zip(vectorizer.get_feature_names_out(), X.toarray().sum(axis=0)))
        arr=[]
        for word in  tfidf_dict:
            if tfidf_dict[word]>thres:
                arr.append(word)
        #print(arr)
        while ind<len(data.news_id)and ind2==data.loc[ind,"news_id"]:
            for word in arr:
                if word in data.loc[ind,"sentence"]:
                    data.loc[ind,"tfidf"]+=1
            ind+=1
        ind2+=1

In [ ]:
def ret_text(ind):
    id=data.loc[ind,"news_id"]
    metin=""
    while ind<len(data.news_id) and data.loc[ind,"news_id"]==id:
        metin+=data.loc[ind,"sentence"]
        ind+=1
    return metin
    

In [ ]:
tfidf(0.3)

In [ ]:
data

,news_id,sentence,words,title,is_contains_idiom,NER,tfidf
0,0.0,Araç sahipleri için pahalı benzine karşı yapıl...,"['Araç', 'sahipleri', 'için', 'pahalı', 'benzi...",Aracınızda yüzde 30 tarassuf edin,-1,0,0
1,0.0,"Shell, Total, BP gibi sektörün önde gelen şirk...","['Shell', 'Total', 'BP', 'gibi', 'sektörün', '...",Aracınızda yüzde 30 tarassuf edin,-1,4,2
2,0.0,Seyir halinde iken gereksiz klima çalıştırmama...,"['Seyir', 'halinde', 'iken', 'gereksiz', 'klim...",Aracınızda yüzde 30 tarassuf edin,-1,1,0
3,0.0,"Daha az yakıt tüketimi, ilk olarak araç bakıml...","['Daha', 'az', 'yakıt', 'tüketimi', 'ilk', 'ol...",Aracınızda yüzde 30 tarassuf edin,-1,0,1
4,0.0,Bunun için motor ve lastik bakımları öne çıkıyor.,"['Bunun', 'için', 'motor', 've', 'lastik', 'ba...",Aracınızda yüzde 30 tarassuf edin,-1,1,0
...,...,...,...,...,...,...,...
143,9.0,Günde 100-150 bin adet üretim yapan endüstriye...,"['Günde', '100150', 'bin', 'adet', 'üretim', '...",İstanbulda 3 fırından birisi ruhsatsız,-1,1,0
144,9.0,diye konuştu.,"['diye', 'konuştu']",İstanbulda 3 fırından birisi ruhsatsız,-1,0,0
145,9.0,Türkiye'de her yıl 12 milyon ton civarında ekm...,"['Türkiyede', 'her', 'yıl', '12', 'milyon', 't...",İstanbulda 3 fırından birisi ruhsatsız,-1,2,1
146,9.0,İstanbul Fırıncılar Odası'nın verdiği rakamlar...,"['İstanbul', 'Fırıncılar', 'Odasının', 'verdiğ...",İstanbulda 3 fırından birisi ruhsatsız,-1,2,1


In [ ]:
dep_parser = DependencyParser()
def dependency_parser():
    data["noun_phrase"]=0
    i=0
    for sentence in data.sentence:
        #print(sentence)
        try:
            parsed=dep_parser.predict(sentence)
            for token in parsed:
                if(token[3]=="nmod"):
                    data.loc[i,"noun_phrase"]+=1
        except Exception:
            a=sentence.split()
            parsed=dep_parser.predict(" ".join(a[0:len(a)//2]))
            for token in parsed:
                if(token[3]=="nmod"):
                    data.loc[i,"noun_phrase"]+=1
            parsed=dep_parser.predict(" ".join(a[len(a)//2:len(a)]))
            for token in parsed:
                if(token[3]=="nmod"):
                    data.loc[i,"noun_phrase"]+=1
        
        i+=1
                

In [ ]:
dependency_parser()

KeyboardInterrupt: ignored

**start from here**

In [37]:
data=dt
data.reset_index(drop=True, inplace=True)
tempo=[]
for i in range(len(data["words2"])):
  temp=[]
  for a in data.loc[i,"words2"].split(","):
    temp.append(a.replace(",","").replace("[","").replace("]","").replace("'","").replace(" ",""))
  tempo.append(temp)
data["words"]=tempo
data=data.drop(columns=["words2"])
data

,news_id,sentence,title,is_contains_idiom,NER,tfidf,noun_phrase,sentence_length,position,result,words
0,1064.0,Nikah yerinizi nasıl seçersiniz?,Evlilik başvurusu nasıl yapılır?,-1,0,0,1,0.010610,0.333333,0.489865,"[Nikah, yerinizi, nasıl, seçersiniz]"
1,1064.0,Kimler nikah şahidi seçilebilir?,Evlilik başvurusu nasıl yapılır?,-1,0,1,1,0.010610,0.250000,0.521200,"[Kimler, nikah, şahidi, seçilebilir]"
2,1064.0,Nikahla ilgili merak ettiğiniz her şey burada…...,Evlilik başvurusu nasıl yapılır?,-1,0,0,0,0.029178,0.200000,0.781842,"[Nikahla, ilgili, merak, ettiğiniz, her, şey, ..."
3,1064.0,Müstakbel eşinizle birlikte yaşadığınız yerdek...,Evlilik başvurusu nasıl yapılır?,-1,1,1,0,0.023873,0.166667,0.657137,"[Müstakbel, eşinizle, birlikte, yaşadığınız, y..."
4,1064.0,"Ancak, ayrı illerde oturuyorsanız kendi illeri...",Evlilik başvurusu nasıl yapılır?,-1,0,1,0,0.031830,0.142857,0.484427,"[Ancak, ayrı, illerde, oturuyorsanız, kendi, i..."
...,...,...,...,...,...,...,...,...,...,...,...
11387,1697.0,"Galatasaray, Gaziantepspor maçının hazırlıklar...",Floryada Lucas Neill rüzgarı,-1,2,0,0,0.042254,0.076923,0.073290,"[Galatasaray, Gaziantepspor, maçının, hazırlık..."
11388,1698.0,"Başbakan Recep Tayyip Erdoğan, özel bir haber ...",Erdoğan bir dava daha kazandı,-1,3,1,3,0.200000,1.000000,0.404898,"[Başbakan, Recep, Tayyip, Erdoğan, özel, bir, ..."
11389,1698.0,Ankara 3.,Erdoğan bir dava daha kazandı,-1,0,0,0,0.013793,0.500000,0.271632,"[Ankara, 3]"
11390,1698.0,Asliye Hukuk Mahkemesi'nde görülen davanın kar...,Erdoğan bir dava daha kazandı,-1,5,1,2,0.117241,0.333333,0.568497,"[Asliye, Hukuk, Mahkemesinde, görülen, davanın..."


In [ ]:

dt


,news_id,sentence,words,title,is_contains_idiom,NER,tfidf,noun_phrase
0,0.0,Araç sahipleri için pahalı benzine karşı yapıl...,"['Araç', 'sahipleri', 'için', 'pahalı', 'benzi...",Aracınızda yüzde 30 tarassuf edin,-1,0,0,1
1,0.0,"Shell, Total, BP gibi sektörün önde gelen şirk...","['Shell', 'Total', 'BP', 'gibi', 'sektörün', '...",Aracınızda yüzde 30 tarassuf edin,-1,4,2,0
2,0.0,Seyir halinde iken gereksiz klima çalıştırmama...,"['Seyir', 'halinde', 'iken', 'gereksiz', 'klim...",Aracınızda yüzde 30 tarassuf edin,-1,1,0,0
3,0.0,"Daha az yakıt tüketimi, ilk olarak araç bakıml...","['Daha', 'az', 'yakıt', 'tüketimi', 'ilk', 'ol...",Aracınızda yüzde 30 tarassuf edin,-1,0,1,3
4,0.0,Bunun için motor ve lastik bakımları öne çıkıyor.,"['Bunun', 'için', 'motor', 've', 'lastik', 'ba...",Aracınızda yüzde 30 tarassuf edin,-1,1,0,0
...,...,...,...,...,...,...,...,...
46142,2500.0,Sizin söylediğiniz şu; Ahmet Tezcan:Sizin kura...,"['Sizin', 'söylediğiniz', 'şu', 'Ahmet', 'Tezc...",Nuray Mert Allah deyip saydırdı,-1,1,1,0
46143,2500.0,Nuray Mert: Olabilirim zaten bende hükümet kur...,"['Nuray', 'Mert', 'Olabilirim', 'zaten', 'bend...",Nuray Mert Allah deyip saydırdı,-1,1,0,0
46144,2500.0,Kurmuyoruz efendim.,"['Kurmuyoruz', 'efendim']",Nuray Mert Allah deyip saydırdı,-1,0,0,0
46145,2500.0,Beni karşılaştırmayın bunları.,"['Beni', 'karşılaştırmayın', 'bunları']",Nuray Mert Allah deyip saydırdı,-1,0,0,0


In [ ]:
sentence=" di götürdüğü topla sağ kanattan ceza sahasına kadar sokuldu ve topu müsait pozisyonda bulunan Makukula'ya verdi, Gaziantepsporlu Zurita dengesini kaybedip düşünce, kaleci ile karşı karşıya kalan Makukula meşin yuvarlağı plase bir vuruşla ağlarla buluşturdu: 0-1."
#a=sentence.split()
#parsed=dep_parser.predict(" ".join(a[0:len(a)//2]))
#parsed+=dep_parser.predict(" ".join(a[len(a)//2:len(a)])) sentence is too long hatası oluşuuyordu
try:
    parsed=dep_parser.predict(sentence)
    print("here")
except:
    print("except")
        

except


In [ ]:
def add_length():
    data["sentence_length"]=0
    for i in range(len(data.words2)):
        data.loc[i,"sentence_length"]=len(data.loc[i,"words2"])

In [ ]:
add_length()


In [ ]:
data

,news_id,sentence,title,is_contains_idiom,NER,tfidf,noun_phrase,words2,sentence_length
0,1400.0,Samantha Millard'ın annesi Debbie Van Horenbee...,Domuz gribi ilacı bu hale getirdi,-1,2,0,0,"[Samantha, Millardın, annesi, Debbie, Van, Hor...",13
1,1400.0,"Samantha'nın annesi Van Horenbeeck, Domuz grib...",Domuz gribi ilacı bu hale getirdi,-1,1,0,0,"[Samanthanın, annesi, Van, Horenbeeck, Domuz, ...",15
2,1400.0,Kızının başından bir an olsun ayrılmayan anne ...,Domuz gribi ilacı bu hale getirdi,-1,1,0,0,"[Kızının, başından, bir, an, olsun, ayrılmayan...",16
3,1400.0,Hükümet Domuz gribi oldugumuzda bize bu ilacı ...,Domuz gribi ilacı bu hale getirdi,-1,1,0,0,"[Hükümet, Domuz, gribi, oldugumuzda, bize, bu,...",10
4,1400.0,"Kızıma Domuz gribi teşhisi koyduktan sonra, bu...",Domuz gribi ilacı bu hale getirdi,-1,0,0,0,"[Kızıma, Domuz, gribi, teşhisi, koyduktan, son...",18
...,...,...,...,...,...,...,...,...,...
19750,2500.0,Sizin söylediğiniz şu; Ahmet Tezcan:Sizin kura...,Nuray Mert Allah deyip saydırdı,-1,1,1,0,"[Sizin, söylediğiniz, şu, Ahmet, TezcanSizin, ...",16
19751,2500.0,Nuray Mert: Olabilirim zaten bende hükümet kur...,Nuray Mert Allah deyip saydırdı,-1,1,0,0,"[Nuray, Mert, Olabilirim, zaten, bende, hüküme...",10
19752,2500.0,Kurmuyoruz efendim.,Nuray Mert Allah deyip saydırdı,-1,0,0,0,"[Kurmuyoruz, efendim]",2
19753,2500.0,Beni karşılaştırmayın bunları.,Nuray Mert Allah deyip saydırdı,-1,0,0,0,"[Beni, karşılaştırmayın, bunları]",3


In [ ]:
def add_sentence_position():
    data["position"]=0.0
    ind=0
    rate=1
    metin_ind=0
    while ind<len(data.news_id):
        if(data.loc[ind,"news_id"]==metin_ind):
            data.loc[ind,"position"]=1/rate
            rate+=1
            ind+=1
        else:
            rate=1
            metin_ind+=1
    

In [ ]:
add_sentence_position()

In [ ]:
data

,Unnamed: 0,news_id,sentence,title,is_contains_idiom,NER,tfidf,noun_phrase,words2,sentence_length,position
0,0,0.0,Araç sahipleri için pahalı benzine karşı yapıl...,Aracınızda yüzde 30 tarassuf edin,-1,0,0,1,"[Araç, sahipleri, için, pahalı, benzine, karşı...",9,1.000000
1,1,0.0,"Shell, Total, BP gibi sektörün önde gelen şirk...",Aracınızda yüzde 30 tarassuf edin,-1,4,2,0,"[Shell, Total, BP, gibi, sektörün, önde, gelen...",32,0.500000
2,2,0.0,Seyir halinde iken gereksiz klima çalıştırmama...,Aracınızda yüzde 30 tarassuf edin,-1,1,0,0,"[Seyir, halinde, iken, gereksiz, klima, çalışt...",16,0.333333
3,3,0.0,"Daha az yakıt tüketimi, ilk olarak araç bakıml...",Aracınızda yüzde 30 tarassuf edin,-1,0,1,3,"[Daha, az, yakıt, tüketimi, ilk, olarak, araç,...",11,0.250000
4,4,0.0,Bunun için motor ve lastik bakımları öne çıkıyor.,Aracınızda yüzde 30 tarassuf edin,-1,1,0,0,"[Bunun, için, motor, ve, lastik, bakımları, ön...",8,0.200000
...,...,...,...,...,...,...,...,...,...,...,...
46142,46142,2500.0,Sizin söylediğiniz şu; Ahmet Tezcan:Sizin kura...,Nuray Mert Allah deyip saydırdı,-1,1,1,0,"[Sizin, söylediğiniz, şu, Ahmet, TezcanSizin, ...",16,0.029412
46143,46143,2500.0,Nuray Mert: Olabilirim zaten bende hükümet kur...,Nuray Mert Allah deyip saydırdı,-1,1,0,0,"[Nuray, Mert, Olabilirim, zaten, bende, hüküme...",10,0.028571
46144,46144,2500.0,Kurmuyoruz efendim.,Nuray Mert Allah deyip saydırdı,-1,0,0,0,"[Kurmuyoruz, efendim]",2,0.027778
46145,46145,2500.0,Beni karşılaştırmayın bunları.,Nuray Mert Allah deyip saydırdı,-1,0,0,0,"[Beni, karşılaştırmayın, bunları]",3,0.027027


In [30]:
dt = pd.read_csv("./data_result_added", sep='\t')


In [38]:
data

,news_id,sentence,title,is_contains_idiom,NER,tfidf,noun_phrase,sentence_length,position,result,words
0,1064.0,Nikah yerinizi nasıl seçersiniz?,Evlilik başvurusu nasıl yapılır?,-1,0,0,1,0.010610,0.333333,0.489865,"[Nikah, yerinizi, nasıl, seçersiniz]"
1,1064.0,Kimler nikah şahidi seçilebilir?,Evlilik başvurusu nasıl yapılır?,-1,0,1,1,0.010610,0.250000,0.521200,"[Kimler, nikah, şahidi, seçilebilir]"
2,1064.0,Nikahla ilgili merak ettiğiniz her şey burada…...,Evlilik başvurusu nasıl yapılır?,-1,0,0,0,0.029178,0.200000,0.781842,"[Nikahla, ilgili, merak, ettiğiniz, her, şey, ..."
3,1064.0,Müstakbel eşinizle birlikte yaşadığınız yerdek...,Evlilik başvurusu nasıl yapılır?,-1,1,1,0,0.023873,0.166667,0.657137,"[Müstakbel, eşinizle, birlikte, yaşadığınız, y..."
4,1064.0,"Ancak, ayrı illerde oturuyorsanız kendi illeri...",Evlilik başvurusu nasıl yapılır?,-1,0,1,0,0.031830,0.142857,0.484427,"[Ancak, ayrı, illerde, oturuyorsanız, kendi, i..."
...,...,...,...,...,...,...,...,...,...,...,...
11387,1697.0,"Galatasaray, Gaziantepspor maçının hazırlıklar...",Floryada Lucas Neill rüzgarı,-1,2,0,0,0.042254,0.076923,0.073290,"[Galatasaray, Gaziantepspor, maçının, hazırlık..."
11388,1698.0,"Başbakan Recep Tayyip Erdoğan, özel bir haber ...",Erdoğan bir dava daha kazandı,-1,3,1,3,0.200000,1.000000,0.404898,"[Başbakan, Recep, Tayyip, Erdoğan, özel, bir, ..."
11389,1698.0,Ankara 3.,Erdoğan bir dava daha kazandı,-1,0,0,0,0.013793,0.500000,0.271632,"[Ankara, 3]"
11390,1698.0,Asliye Hukuk Mahkemesi'nde görülen davanın kar...,Erdoğan bir dava daha kazandı,-1,5,1,2,0.117241,0.333333,0.568497,"[Asliye, Hukuk, Mahkemesinde, görülen, davanın..."


In [44]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return len(lst3)
def union(lst1, lst2):
    final_list = lst1 + lst2
    return len(final_list)

In [42]:
def title_similarity():
    data["result_old"]=0.0
    for i in range(len(data.sentence)):
        title=stopword_remover.drop_stop_words(nltk.word_tokenize(data.loc[i,"title"].translate(translator).replace("“","").replace("’","")))
        similarity=intersection(data.loc[i,"words"],title)/union(data.loc[i,"words"],title)
        data.loc[i,"result_old"]=similarity

In [45]:
title_similarity()
data

,news_id,sentence,title,is_contains_idiom,NER,tfidf,noun_phrase,sentence_length,position,result,words,result_old
0,1064.0,Nikah yerinizi nasıl seçersiniz?,Evlilik başvurusu nasıl yapılır?,-1,0,0,1,0.010610,0.333333,0.000000,"[Nikah, yerinizi, nasıl, seçersiniz]",0.000000
1,1064.0,Kimler nikah şahidi seçilebilir?,Evlilik başvurusu nasıl yapılır?,-1,0,1,1,0.010610,0.250000,0.000000,"[Kimler, nikah, şahidi, seçilebilir]",0.000000
2,1064.0,Nikahla ilgili merak ettiğiniz her şey burada…...,Evlilik başvurusu nasıl yapılır?,-1,0,0,0,0.029178,0.200000,0.000000,"[Nikahla, ilgili, merak, ettiğiniz, her, şey, ...",0.000000
3,1064.0,Müstakbel eşinizle birlikte yaşadığınız yerdek...,Evlilik başvurusu nasıl yapılır?,-1,1,1,0,0.023873,0.166667,0.000000,"[Müstakbel, eşinizle, birlikte, yaşadığınız, y...",0.000000
4,1064.0,"Ancak, ayrı illerde oturuyorsanız kendi illeri...",Evlilik başvurusu nasıl yapılır?,-1,0,1,0,0.031830,0.142857,0.000000,"[Ancak, ayrı, illerde, oturuyorsanız, kendi, i...",0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
11387,1697.0,"Galatasaray, Gaziantepspor maçının hazırlıklar...",Floryada Lucas Neill rüzgarı,-1,2,0,0,0.042254,0.076923,0.000000,"[Galatasaray, Gaziantepspor, maçının, hazırlık...",0.000000
11388,1698.0,"Başbakan Recep Tayyip Erdoğan, özel bir haber ...",Erdoğan bir dava daha kazandı,-1,3,1,3,0.200000,1.000000,0.090909,"[Başbakan, Recep, Tayyip, Erdoğan, özel, bir, ...",0.090909
11389,1698.0,Ankara 3.,Erdoğan bir dava daha kazandı,-1,0,0,0,0.013793,0.500000,0.000000,"[Ankara, 3]",0.000000
11390,1698.0,Asliye Hukuk Mahkemesi'nde görülen davanın kar...,Erdoğan bir dava daha kazandı,-1,5,1,2,0.117241,0.333333,0.000000,"[Asliye, Hukuk, Mahkemesinde, görülen, davanın...",0.000000


In [48]:
np.sum(data.result_old)/11391

0.009948168623703136

In [ ]:
text_length = np.zeros(2501)
for i in range(len(data["news_id"])):
  text_length[int(data.loc[i,"news_id"])] +=  data.loc[i,"sentence_length"]

In [ ]:
for i in range(len(data["news_id"])):
  data.loc[i,"sentence_length"]=data.loc[i,"sentence_length"]/text_length[int(data.loc[i,"news_id"])]

In [ ]:
data["sentence_length"].quantile([.000001, .5,.99])

0.000001    0.000460
0.500000    0.031700
0.990000    0.325581
Name: sentence_length, dtype: float64

In [ ]:
data.to_csv("data_son_other",sep="\t")

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.svm import SVR
import pickle
import math
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [ ]:
X_tr, X_ts, y_train, y_test = train_test_split(data.loc[:,["news_id","sentence","title","is_contains_idiom","NER","tfidf","sentence_length","position"]],  data.loc[:,"result"], test_size=0.33, random_state=47)

In [ ]:
import random

# Initialize an empty set to store unique numbers
unique_numbers = set()

# Generate random numbers until we have 2000 unique numbers
while len(unique_numbers) < 2000:
    # Generate a random number between 0 and 2500
    random_number = random.randint(0, 2500)
    # Add the number to the set if it's not already in it
    unique_numbers.add(random_number)

# Convert the set to a list and sort it
unique_numbers_list = sorted(list(unique_numbers))

In [ ]:
X_rand_train=[]
X_rand_test=[]
for i in range(len(data)):
  if data.loc[i,"news_id"] in unique_numbers_list:
    X_rand_train.append(data.loc[i,:])
  else:
    X_rand_test.append(data.loc[i,:])

In [ ]:
random_train = pd.DataFrame (X_rand_train, columns = data.columns)
random_test = pd.DataFrame (X_rand_test, columns = data.columns)

In [ ]:
print((y_train.index))

Int64Index([36351, 28698,  7347, 35572, 42887, 40231,  2123, 21466, 13232,
             9742,
            ...
             3185, 35501, 26695, 15056, 39995, 35863, 25267, 23112, 11528,
            38023],
           dtype='int64', length=30918)


In [ ]:
X_tr=random_train.loc[:,["news_id","sentence","title","is_contains_idiom","NER","tfidf","sentence_length","position"]]
X_ts=random_test.loc[:,["news_id","sentence","title","is_contains_idiom","NER","tfidf","sentence_length","position"]]
y_train=random_train.loc[:,["result"]]
for i in y_train.index:
  if y_train.loc[i,"result"] != 0 :
    y_train.loc[i,"result"]=math.log(y_train.loc[i,"result"])*-1
y_test=random_test.loc[:,["result"]]
for i in y_test.index:
  if y_test.loc[i,"result"] != 0 :
    y_test.loc[i,"result"]=math.log(y_test.loc[i,"result"])*-1

In [ ]:
y_train.loc[0,"result"]

0.0

In [ ]:
X_train=X_tr.loc[:,["is_contains_idiom","NER","tfidf","sentence_length","position"]]
X_test=X_ts.loc[:,["is_contains_idiom","NER","tfidf","sentence_length","position"]]

In [ ]:
regressor = RandomForestRegressor(n_estimators = 600, random_state = 0)
regressor.fit(X_train,y_train) 

<ipython-input-103-ec162e6ddcc8>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regressor.fit(X_train,y_train)


RandomForestRegressor(n_estimators=600, random_state=0)

In [ ]:
Y_pred = regressor.predict(X_test)

In [ ]:
m1=mean_squared_error(y_test,Y_pred)
m2=mean_absolute_error(y_test,Y_pred)
print(m1,m2)

1.0796369114139301 0.6248540412096087


In [ ]:
clf = linear_model.Lasso(alpha=0.1)
clf.fit(X_train,y_train) 


Lasso(alpha=0.1)

In [ ]:
y_pred2=clf.predict(X_test)

In [ ]:
mL1=mean_squared_error(y_test,y_pred2)
mL2=mean_absolute_error(y_test,y_pred2)
print(mL1,mL2)

1.0497120422012094 0.7220508402653534


In [ ]:
clfSVR = regressor = SVR(kernel='poly')
clfSVR.fit(X_train,y_train) 

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
y_pred3=clfSVR.predict(X_test)
mSVR1=mean_squared_error(y_test,y_pred3)
mSVR2=mean_absolute_error(y_test,y_pred3)
print(mSVR1,mSVR2)

NameError: ignored

In [ ]:
pickle.dump(regressor, open("regression.sav", 'wb'))
pickle.dump(clf, open("lasso.sav", 'wb'))


In [ ]:
random_test["y_pred"]=Y_pred
random_test["result"]=y_test
random_test

,Unnamed: 0,news_id,sentence,title,is_contains_idiom,NER,tfidf,noun_phrase,words2,sentence_length,position,result,y_pred
13,13,1.0,"İki insanın kaderini birleştiren olay, Adana S...",Kahraman öğretmen hayat kurtardı,-1,1,0,0,"[İki, insanın, kaderini, birleştiren, olay, Ad...",0.086022,1.000000,0.000000,0.472452
14,14,1.0,"Nehrin üzerindeki köprüden geçen Döndü Akçan, ...",Kahraman öğretmen hayat kurtardı,-1,1,0,2,"[Nehrin, üzerindeki, köprüden, geçen, Döndü, A...",0.139785,0.500000,0.000000,0.696153
15,15,1.0,Başı döndüğü için doktora gitmekte olduğunu sö...,Kahraman öğretmen hayat kurtardı,-1,1,0,0,"[Başı, döndüğü, için, doktora, gitmekte, olduğ...",0.129032,0.333333,0.000000,1.485140
16,16,1.0,Akçan’ın nehre düşüşünü oradan geçmekte olan g...,Kahraman öğretmen hayat kurtardı,-1,2,0,2,"[Akçan, ’, ın, nehre, düşüşünü, oradan, geçmek...",0.150538,0.250000,2.890372,0.375055
17,17,1.0,Cenk arabasını durdurup hemen koşarak nehre at...,Kahraman öğretmen hayat kurtardı,-1,1,0,1,"[Cenk, arabasını, durdurup, hemen, koşarak, ne...",0.075269,0.200000,0.000000,0.831866
...,...,...,...,...,...,...,...,...,...,...,...,...,...
46142,46142,2500.0,Sizin söylediğiniz şu; Ahmet Tezcan:Sizin kura...,Nuray Mert Allah deyip saydırdı,-1,1,1,0,"[Sizin, söylediğiniz, şu, Ahmet, TezcanSizin, ...",0.039024,0.029412,0.000000,0.911351
46143,46143,2500.0,Nuray Mert: Olabilirim zaten bende hükümet kur...,Nuray Mert Allah deyip saydırdı,-1,1,0,0,"[Nuray, Mert, Olabilirim, zaten, bende, hüküme...",0.024390,0.028571,2.014903,0.077637
46144,46144,2500.0,Kurmuyoruz efendim.,Nuray Mert Allah deyip saydırdı,-1,0,0,0,"[Kurmuyoruz, efendim]",0.004878,0.027778,0.000000,0.000000
46145,46145,2500.0,Beni karşılaştırmayın bunları.,Nuray Mert Allah deyip saydırdı,-1,0,0,0,"[Beni, karşılaştırmayın, bunları]",0.007317,0.027027,0.000000,0.000000


In [ ]:
random 

In [3]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 44.1 MB/s eta 0:00:00


In [4]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

model = AutoModelForSequenceClassification.from_pretrained("savasy/bert-base-turkish-sentiment-cased")
tokenizer = AutoTokenizer.from_pretrained("savasy/bert-base-turkish-sentiment-cased")
sa= pipeline("sentiment-analysis", tokenizer=tokenizer, model=model)

p = sa("bu telefon modelleri çok kaliteli , her parçası çok özel bence")
print(p)
# [{'label': 'LABEL_1', 'score': 0.9871089}]
print(p[0]['label'] == 'LABEL_1')
# True

p = sa("Film çok kötü ve çok sahteydi")
print(p)
# [{'label': 'LABEL_0', 'score': 0.9975505}]
print(p[0]['label'] == 'LABEL_1')
# False

[{'label': 'positive', 'score': 0.983146607875824}]
False
[{'label': 'negative', 'score': 0.9990053772926331}]
False
